In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

cuda


In [11]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)

In [12]:
string_to_int = {ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [13]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[73, 68,  1, 62, 73, 11,  0,  0],
        [ 1, 76, 62, 67, 57,  1, 66, 54],
        [62, 71,  1, 54, 67, 57,  1, 69],
        [61, 62, 72, 10, 10, 62, 59,  1]], device='cuda:0')
targets:
tensor([[68,  1, 62, 73, 11,  0,  0, 44],
        [76, 62, 67, 57,  1, 66, 54, 57],
        [71,  1, 54, 67, 57,  1, 69, 74],
        [62, 72, 10, 10, 62, 59,  1, 76]], device='cuda:0')


In [14]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [16]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


:FemTO a.M2WSFD1TWv!AF(2j3QVl 7]t*kvx4 1zh"R*Mk﻿,ITub*2
.yZHtOihyI"kk﻿IV](hzrFj21h-E﻿b-WPgydW5-whRe3w4;M6S:qa'-3,1Q)D﻿mF_3F&mEM!HR!MrfP[vc*n qvN6JtqDK6dWSn]D﻿)]RU_V8cI"jeLnRy9PEjspuZ6EjZsV]GH;__GcN 4)d4[IhFB;
.kOP:h7Kql*R!dwctV7X-("?0VHqw4civA.r;9[]ulaN2V]hcQ0bZ_yJjJNPC cQTuHq7K7EmYmP40)j( 
v
4&s"g5-'z
OqdbAF vwc55]]qRw1&jd*pEljmaiVW221fB5RWuH Lnd8i'f]
*nRGgLhI]?
*g91wJqA&I0) aJLCZ2D,!tD 2wF)Www)o(B)sivH-﻿0v5sp﻿)Cf6dq7hbHFc)Yt!XM0rn[n60uWmV[Qa,IoIeWsYm
obs8eQM.Z﻿_:uUp﻿knwF_Z7hF?w7[n:7J)Zxo2SwcQR


In [17]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.877, val loss: 4.877
step: 250, train loss: 4.803, val loss: 4.784
step: 500, train loss: 4.727, val loss: 4.742
step: 750, train loss: 4.690, val loss: 4.668
4.782811641693115


In [18]:
context= torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


-ELjbAE9El[RY6mUJqxoZ30a?LFt94 859lKt*Oi0Wm
rfBI"?,mS6UgtEl8)MS[z5!;bL(t﻿4LG&O.zA):REghem&A8OGHj1SMTUEXb]q pCrpu3UE'ZTOr8M2hyhI4[R;9o;bi-HRAd&;3l[.DyJ52W"l&xZnpqC"RWvIf,e92y9[e80_G

rb]5YPofhs(Pm2WrqU4rQ2I2Wt4S) -B8WGv0"K."X:p?c9Z;TUgW:wtko"R1Ym9pu31w,GX6f52zAd',S[UWUJ5gV&oo9UMk8]1w,Yaai(SrREPe6d2WLTF1f](iO.q9JquF5fO5MPcHqMg7VBgh﻿QT

TOs﻿b'Z78qxc6[BK(4Vib1FkI:gh[I[U7VGViW*MT(mtj*0XDXQTh_
T*&h&LN7J[RNhH;f)Ut hs[x2WrnR1L
V﻿s6qDnAd
*r2)&lCg[Wy_e6Cy4s!Z,)71cmkGkC.I"!dQZ7Lnqk]Cb,J[.E:z[Z68]RWw4L?rf6[
